In [ ]:
# ==============================================================================
# SEÇÃO 0: SETUP E INICIALIZAÇÃO
# ==============================================================================
# Este bloco configura o ambiente, importa as bibliotecas necessárias e
# inicializa o AnalisadorBancario, deixando-o pronto para uso.
# ------------------------------------------------------------------------------
print("--- INICIANDO SETUP E INICIALIZAÇÃO ---")

import pandas as pd
import numpy as np
from pathlib import Path
import sys
import matplotlib.pyplot as plt

# Adiciona o diretório 'Code' ao path para encontrar o DataUtils
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

from DataUtils import AnalisadorBancario

# Configurações de exibição do Pandas e Matplotlib
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')

# Inicializa o Analisador (carrega todos os dados em memória)
output_dir = Path('..').resolve() / 'Output'
analisador = AnalisadorBancario(diretorio_output=str(output_dir))

print("\n--- SETUP CONCLUÍDO. ANALISADOR PRONTO PARA USO. ---")

In [ ]:
# ==============================================================================
# SEÇÃO 1: DEMONSTRAÇÕES DE CONSULTAS BÁSICAS
# ==============================================================================
# Esta seção mostra como usar os métodos fundamentais para extrair dados
# de cada uma das fontes principais.
# ------------------------------------------------------------------------------

# --- 1.1: Buscando dados contábeis da fonte COSIF ---
print("\n--- 1.1: Consulta básica de dados COSIF ---")
dados_cosif = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=['ATIVO REALIZÁVEL', 'Caixa'],
    datas=202403,
    documentos=4060 # Filtra pelo balancete prudencial
)
print("Resultado para 'BCO DO BRASIL S.A.' (COSIF):")
display(dados_cosif)
print("-" * 80)


# --- 1.2: Buscando indicadores regulatórios da fonte IF.DATA ---
print("\n--- 1.2: Consulta básica de dados IF.DATA ---")
dados_ifdata = analisador.get_dados_ifdata(
    identificador='28195667', # CNPJ do Banco ABC Brasil
    contas=[78282, 78283, 78284], # Códigos para Depósitos à Vista, Poupança, etc.
    datas=202403
)
print("Resultado para 'BANCO ABC BRASIL S.A.' (IF.DATA):")
display(dados_ifdata)
print("-" * 80)


# --- 1.3: Buscando atributos do cadastro das instituições ---
print("\n--- 1.3: Consulta de atributos do cadastro ---")
atributos_cadastro = analisador.get_atributos_cadastro(
    identificador=['ITAU', 'BRADESCO', 'NUBANK'],
    atributos=['TCB_IFD_CAD', 'SITUACAO_IFD_CAD', 'SR_IFD_CAD']
)
print("Resultado para atributos de cadastro:")
display(atributos_cadastro)
print("-" * 80)

In [ ]:
# ==============================================================================
# SEÇÃO 2: DEMONSTRAÇÕES DE ANÁLISE AGREGADA
# ==============================================================================
# Esta seção mostra os métodos de alto nível para comparar instituições
# e analisar a evolução de indicadores ao longo do tempo.
# ------------------------------------------------------------------------------

# --- 2.1: Criando uma tabela comparativa de indicadores ---
print("\n--- 2.1: Tabela comparativa de indicadores (Pares) ---")
indicadores = {
    'Patrimônio Líquido': {'tipo': 'COSIF', 'conta': 'PATRIMÔNIO LÍQUIDO'},
    'Total Ativos': {'tipo': 'COSIF', 'conta': 'TOTAL GERAL DO ATIVO'},
    'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia'},
    'Situação': {'tipo': 'Atributo', 'atributo': 'SITUACAO_IFD_CAD'}
}
bancos = ['61190658', 'BANCO BRADESCO', 'NU FINANCEIRA', 'BANCO BTG PACTUAL', 'BCO DO BRASIL S.A.']

tabela_comparativa = analisador.comparar_indicadores(
    identificadores=bancos,
    indicadores=indicadores,
    data=202403,
    documento_cosif=4060 
)
print("Comparativo de indicadores para os principais bancos:")
display(tabela_comparativa)
print("-" * 80)


# --- 2.2: Gerando uma série temporal e plotando um gráfico ---
print("\n--- 2.2: Série temporal de um indicador ---")
serie_temporal = analisador.get_serie_temporal_indicador(
    identificador='NU FINANCEIRA',
    conta='TOTAL GERAL DO ATIVO',
    data_inicio=202401,
    data_fim=202412,
    fonte='COSIF'
)
print("Série Temporal para 'TOTAL GERAL DO ATIVO' do Nubank:")
display(serie_temporal)

# Plotando o resultado
if not serie_temporal.empty:
    fig, ax = plt.subplots(figsize=(12, 6))
    serie_temporal.plot(ax=ax, marker='o', linestyle='-', title="Evolução do Ativo Total - Nubank")
    ax.set_ylabel("Valor (em R$)")
    ax.set_xlabel("Data")
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x/1e9:,.0f} bi')) # Formata em bilhões
    plt.tight_layout()
    plt.show()
print("-" * 80)

In [ ]:
# ==============================================================================
# SEÇÃO 3: TÉCNICAS AVANÇADAS E CASOS DE BORDA
# ==============================================================================
# Esta seção demonstra a flexibilidade e robustez da classe em cenários
# mais complexos e no tratamento de erros.
# ------------------------------------------------------------------------------

# --- 3.1: Consulta com lista mista de nomes e códigos de contas ---
print("\n--- 3.1: Flexibilidade de consulta (nomes e códigos de contas) ---")
contas_mistas = [
    'TOTAL GERAL DO ATIVO', # Busca por nome
    60000002                # Busca pelo código do 'PATRIMÔNIO LÍQUIDO'
]
dados_mistos = analisador.get_dados_cosif(
    identificador='BANCO BRADESCO S.A.',
    contas=contas_mistas,
    datas=202403,
    documentos=4060
)
print("Resultado da busca mista para 'BANCO BRADESCO S.A.':")
display(dados_mistos[['Nome_Entidade', 'CONTA_COSIF', 'NOME_CONTA_COSIF', 'VALOR_CONTA_COSIF']])
print("-" * 80)


# --- 3.2: Comparando dados Individuais vs. Prudenciais ---
print("\n--- 3.2: Diferença entre dados Individuais e Prudenciais ---")
conta_unica = 'TOTAL GERAL DO ATIVO'
dado_prudencial = analisador.get_dados_cosif(identificador='BCO DO BRASIL S.A.', contas=[conta_unica], datas=202403, tipo='prudencial')
dado_individual = analisador.get_dados_cosif(identificador='BCO DO BRASIL S.A.', contas=[conta_unica], datas=202403, tipo='individual')

print(f"Ativo Total (Prudencial): {dado_prudencial['VALOR_CONTA_COSIF'].iloc[0]:,.2f}")
print(f"Ativo Total (Individual): {dado_individual['VALOR_CONTA_COSIF'].iloc[0]:,.2f}")
print("-" * 80)


# --- 3.3: Tratamento de Zeros e Nulos com `fillna` ---
print("\n--- 3.3: Flexibilidade no tratamento de nulos (fillna) ---")
banco_serie = 'BANCO INTER'
conta_serie = 'Letras de Crédito Imobiliário (c1)'
print(f"Série para '{conta_serie}' do {banco_serie} (Padrão: com 0 e NaN)")
serie_padrao = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, 
    conta=conta_serie, 
    data_inicio=202401, 
    data_fim=202406, 
    fonte='IFDATA'
    )
display(serie_padrao)

print(f"\nMesma série (preenchendo ausentes com 0)")
serie_fill_zero = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, 
    conta=conta_serie, 
    data_inicio=202401, 
    data_fim=202406, 
    fonte='IFDATA', 
    fillna=0)
display(serie_fill_zero)

print(f"\nMesma série (tratando 0 como ausente)")
serie_zeros_nan = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, 
    conta=conta_serie, 
    data_inicio=202401, 
    data_fim=202406, 
    fonte='IFDATA', 
    fillna=np.nan)
display(serie_zeros_nan)
print("-" * 80)


# --- 3.4: Tratamento de inputs inválidos ---
print("\n--- 3.4: Robustez contra inputs inválidos ---")
identificadores_invalidos = ['BANCO QUE NAO EXISTE', '99999999']
contas_invalidas = {'Indicador Inexistente': {'tipo': 'COSIF', 'conta': 'CONTA QUE NAO EXISTE'}}

tabela_invalida = analisador.comparar_indicadores(
    identificadores=identificadores_invalidos,
    indicadores=contas_invalidas,
    data=202403
)
print("Resultado da tabela com inputs inválidos (esperado: tabela com erros e Nones):")
display(tabela_invalida)
print("-" * 80)


# --- 3.5: Robustez das fontes (busca cruzada) ---
print("\n--- 3.5: Robustez na separação de fontes ---")
dados_cruzados = analisador.get_dados_ifdata(
    identificador='BCO DO BRASIL S.A.',
    contas=['ATIVO REALIZÁVEL'],
    datas=202403
)
print(f"Buscando conta COSIF na fonte IFDATA. Linhas encontradas: {len(dados_cruzados)} (esperado: 0).")
display(dados_cruzados)

print("\n\n--- FIM DAS DEMONSTRAÇÕES ---")